In [65]:
# Importing all necessary packages from Keras Applications API 
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint
import sys
import keras
from PIL import Image
sys.modules['Image'] = Image 

In [66]:
keras.backend.clear_session()

In [67]:
# Parameters 
set_seed = 42
num_classes = 6
batch_size = 16
epochs = 40
patience_epochs = 5
train_val_dir = './split_smoking_images/train/'
test_dir = './split_smoking_images/test/'

In [68]:
# Create the base pre-trained model, without top dense layers
base_model = MobileNetV2(weights='imagenet', include_top=False)
# We can see all the layers: 
#base_model.summary()

In [69]:
x = base_model.output
# Add a global spatial average pooling layer to reduce dimensionality.
x = GlobalAveragePooling2D()(x)
# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# And a logistic layer of width num_classes 
predictions = Dense(num_classes, activation='softmax')(x)
# Complete model using Model object
model = Model(inputs=base_model.input, outputs=predictions)

In [70]:
# first: train only the top layers (randomly initialized)
# i.e. freeze all convolutional MobileNetV2 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.metrics_names)

['loss', 'acc']


In [71]:
train_val_datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15, rescale=1./255,
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest",
                         validation_split=0.2) # set validation split

train_generator = train_val_datagen.flow_from_directory(
    train_val_dir, batch_size=batch_size, shuffle=True, seed=set_seed,
    subset='training') # set as training data to seperate from validation!

val_generator = train_val_datagen.flow_from_directory(
    train_val_dir, batch_size=batch_size, shuffle=True, seed=set_seed,
    subset='validation') # set as validation data to seperate from training!

print(train_generator.class_indices)

Found 2801 images belonging to 6 classes.
Found 697 images belonging to 6 classes.
{'beer_bottle': 0, 'beer_glass': 1, 'grocery_store': 2, 'library': 3, 'pew': 4, 'tobacco_store': 5}


In [72]:
# Callbacks 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=patience_epochs)
checkpoint_callback = ModelCheckpoint('mobilenetv2'+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [73]:
# Training
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator, 
    validation_steps = val_generator.samples // batch_size,
    epochs = epochs,
    callbacks=[early_stopping_callback, checkpoint_callback], verbose=1
)

Epoch 1/40
175/175 [==============================] - 51s 290ms/step - loss: 0.8507 - acc: 0.7029 - val_loss: 0.5491 - val_acc: 0.7820

Epoch 00001: val_loss improved from inf to 0.54910, saving model to mobilenetv2.h5
Epoch 2/40
175/175 [==============================] - 47s 270ms/step - loss: 0.5912 - acc: 0.7939 - val_loss: 0.4852 - val_acc: 0.8179

Epoch 00002: val_loss improved from 0.54910 to 0.48524, saving model to mobilenetv2.h5
Epoch 3/40
175/175 [==============================] - 50s 285ms/step - loss: 0.5231 - acc: 0.8211 - val_loss: 0.6573 - val_acc: 0.7988

Epoch 00003: val_loss did not improve from 0.48524
Epoch 4/40
175/175 [==============================] - 50s 284ms/step - loss: 0.5170 - acc: 0.8129 - val_loss: 0.4670 - val_acc: 0.8326

Epoch 00004: val_loss improved from 0.48524 to 0.46700, saving model to mobilenetv2.h5
Epoch 5/40
175/175 [==============================] - 52s 297ms/step - loss: 0.5014 - acc: 0.8250 - val_loss: 0.4373 - val_acc: 0.8502

Epoch 00005:

In [74]:
# Make a final data generator, for evaluating.
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = train_val_datagen.flow_from_directory(
    test_dir, batch_size=400, shuffle=False) 

loss, acc = model.evaluate_generator(test_generator, verbose=1, steps = 2,
    use_multiprocessing=True)

Found 880 images belonging to 6 classes.
2/2 [==============================] - 14s 7s/step


In [75]:
print("The test loss is " + str(loss))
print("The test accuracy is " + str(acc))

The test loss is 0.4353136122226715
The test accuracy is 0.8537499904632568


In [76]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 Conv1_pad
2 Conv1
3 bn_Conv1
4 Conv1_relu
5 expanded_conv_depthwise
6 expanded_conv_depthwise_BN
7 expanded_conv_depthwise_relu
8 expanded_conv_project
9 expanded_conv_project_BN
10 block_1_expand
11 block_1_expand_BN
12 block_1_expand_relu
13 block_1_pad
14 block_1_depthwise
15 block_1_depthwise_BN
16 block_1_depthwise_relu
17 block_1_project
18 block_1_project_BN
19 block_2_expand
20 block_2_expand_BN
21 block_2_expand_relu
22 block_2_depthwise
23 block_2_depthwise_BN
24 block_2_depthwise_relu
25 block_2_project
26 block_2_project_BN
27 block_2_add
28 block_3_expand
29 block_3_expand_BN
30 block_3_expand_relu
31 block_3_pad
32 block_3_depthwise
33 block_3_depthwise_BN
34 block_3_depthwise_relu
35 block_3_project
36 block_3_project_BN
37 block_4_expand
38 block_4_expand_BN
39 block_4_expand_relu
40 block_4_depthwise
41 block_4_depthwise_BN
42 block_4_depthwise_relu
43 block_4_project
44 block_4_project_BN
45 block_4_add
46 block_5_expand
47 block_5_expand_BN
48 block_5_exp

In [77]:
# we chose to train the top 2 blocks, i.e. we will freeze
# the first 135 layers and unfreeze the rest:
for layer in model.layers[:144]:
   layer.trainable = False
for layer in model.layers[144:]:
   layer.trainable = True

In [78]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.metrics_names)

['loss', 'acc']


In [79]:
# Callbacks 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=patience_epochs)
checkpoint_callback = ModelCheckpoint('mobilenetv2'+'.h6', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [80]:
# Training
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator, 
    validation_steps = val_generator.samples // batch_size,
    epochs = epochs,
    callbacks=[early_stopping_callback, checkpoint_callback], verbose=1
)

Epoch 1/40
175/175 [==============================] - 55s 313ms/step - loss: 0.7399 - acc: 0.7436 - val_loss: 1.3030 - val_acc: 0.7342

Epoch 00001: val_loss improved from inf to 1.30303, saving model to mobilenetv2.h6
Epoch 2/40
175/175 [==============================] - 51s 289ms/step - loss: 0.5353 - acc: 0.8150 - val_loss: 1.1632 - val_acc: 0.7254

Epoch 00002: val_loss improved from 1.30303 to 1.16325, saving model to mobilenetv2.h6
Epoch 3/40
175/175 [==============================] - 51s 289ms/step - loss: 0.4551 - acc: 0.8425 - val_loss: 0.5828 - val_acc: 0.8517

Epoch 00003: val_loss improved from 1.16325 to 0.58282, saving model to mobilenetv2.h6
Epoch 4/40
175/175 [==============================] - 51s 293ms/step - loss: 0.4381 - acc: 0.8497 - val_loss: 1.0170 - val_acc: 0.7739

Epoch 00004: val_loss did not improve from 0.58282
Epoch 5/40
175/175 [==============================] - 51s 290ms/step - loss: 0.3813 - acc: 0.8636 - val_loss: 1.3777 - val_acc: 0.7313

Epoch 00005:

In [81]:
loss, acc = model.evaluate_generator(test_generator, verbose=1, steps = 2)

2/2 [==============================] - 12s 6s/step


In [82]:
print("The test loss is " + str(loss))
print("The test accuracy is " + str(acc))

The test loss is 0.8223159611225128
The test accuracy is 0.8362500071525574
